# Lucia

In [9]:
import json
from aries_askar import Key, KeyAlg
from didcomm_messaging.crypto.backend.askar import AskarCryptoService, AskarSecretKey
from didcomm_messaging.crypto.backend.basic import InMemorySecretsManager
from didcomm_messaging.packaging import PackagingService
from didcomm_messaging.multiformats import multibase
from didcomm_messaging.multiformats import multicodec
from didcomm_messaging.resolver.peer import Peer2, Peer4
from didcomm_messaging.resolver import PrefixResolver
from did_peer_2 import KeySpec, generate, json
from didcomm_messaging import DIDCommMessaging
from didcomm_messaging.routing import RoutingService
from pydid.did import DID


# Define DIDComm Messaging Service Endpoint

In [10]:
didcomm_host = "localhost"
didcomm_port = 8767
didcomm_messaging_uri = f"ws://{didcomm_host}:{didcomm_port}"

# Create Coordinator DID

In [11]:
secrets = InMemorySecretsManager()
crypto = AskarCryptoService()
verkey = Key.generate(KeyAlg.ED25519)
xkey = Key.generate(KeyAlg.X25519)
did = generate(
    [
        KeySpec.verification(
            multibase.encode(
                multicodec.wrap("ed25519-pub", verkey.get_public_bytes()),
                "base58btc",
            )
        ),
        KeySpec.key_agreement(
            multibase.encode(
                multicodec.wrap("x25519-pub", xkey.get_public_bytes()), "base58btc"
            )
        ),
    ],
    [
                {
                    "type": "DIDCommMessaging",
                    "serviceEndpoint": {
                        "uri": didcomm_messaging_uri,
                        "accept": ["didcomm/v2"],
                        "routingKeys": [],
                    },
                },
    ],
)
await secrets.add_secret(AskarSecretKey(verkey, f"{did}#key-1"))
await secrets.add_secret(AskarSecretKey(xkey, f"{did}#key-2"))


In [12]:
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})

did_document = await resolver.resolve(did)
print(json.dumps(did_document, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://w3id.org/security/multikey/v1"
  ],
  "id": "did:peer:2.Vz6MkiN6uhd8HP4RCyuYsQLg3ysWEAej1hkRHk9miJsKmx4xe.Ez6LSdkAdh4dmeBcFciCR71qDRrivNAEegRTDYANGCeDat6YH.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjciLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "verificationMethod": [
    {
      "type": "Multikey",
      "id": "#key-1",
      "controller": "did:peer:2.Vz6MkiN6uhd8HP4RCyuYsQLg3ysWEAej1hkRHk9miJsKmx4xe.Ez6LSdkAdh4dmeBcFciCR71qDRrivNAEegRTDYANGCeDat6YH.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjciLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
      "publicKeyMultibase": "z6MkiN6uhd8HP4RCyuYsQLg3ysWEAej1hkRHk9miJsKmx4xe"
    },
    {
      "type": "Multikey",
      "id": "#key-2",
      "controller": "did:peer:2.Vz6MkiN6uhd8HP4RCyuYsQLg3ysWEAej1hkRHk9miJsKmx4xe.Ez6LSdkAdh4dmeBcFciCR71qDRrivNAEegRTDYANGCeDat6YH.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjciLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",

# Instantiate DIDComm Messaging 

This is used to pack and unpack DIDComm messages.

In [13]:
packaging = PackagingService()
router = RoutingService()

dmp = DIDCommMessaging(
    crypto=crypto,
    secrets=secrets,
    resolver=resolver,
    packaging=packaging,
    routing=router,
)

# Create send message functon

Sends packed DIDComm messages to a websocket endpoint

In [14]:
async def send_message(endpoint, packed):
    async with websockets.connect(endpoint) as websocket:
        message = "Hello from the client!"
        print(f"Sending message: {packed}")
        await websocket.send(packed)  # Send message to the server
        
        response = await websocket.recv()  # Wait for the server's response
        print(f"Received response: {response}")

# Setup DIDComm Messaging Websocket

This is where DIDComm messages are sent to the coordinator

In [15]:
import asyncio
import websockets

received_messages = []


# This handler will be called for every client connection.
async def handle_messages(websocket):
    print("Client connected")
    try:
        async for message in websocket:
            print(f"Received: {message}")
            # Echo the received message back to the client.
            # await websocket.send(f"Echo: {message}")

            unpacked =  await dmp.packaging.unpack(crypto, resolver, secrets,message)
            received_messages.append(unpacked)
            print(f"Unpacked Message : {unpacked}")
    except websockets.exceptions.ConnectionClosed as e:
        print("Connection closed", e)

# The main function to start the server.
async def run_websocket():
    # Start the server on localhost at port 8765.
    async with websockets.serve(handle_messages, didcomm_host, didcomm_port):
        print(f"WebSocket server started on {didcomm_messaging_uri}")
        # Run the server until it is manually stopped.
        await asyncio.Future()  # Run forever

In [16]:
asyncio.create_task(run_websocket())

<Task pending name='Task-11' coro=<run_websocket() running at /tmp/ipykernel_45484/2569901432.py:23>>